In [1]:
spark
import numpy as np
import pandas as pd
import re
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import trim,ltrim,rtrim
from pyspark.sql.functions import *
from pyspark.sql.functions import mean
from ydata_profiling import ProfileReport
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import lit
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.sql.functions import mean, when, col
from pyspark.ml.classification import DecisionTreeClassificationModel
from pycaret.classification import *
from pycaret.classification import ClassificationExperiment
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

Intitializing PySpark ...

23/06/01 20:42:31 WARN Utils: Your hostname, DESKTOP-VJTDOFP resolves to a loopback address: 127.0.1.1; using 172.27.220.32 instead (on interface eth0)
23/06/01 20:42:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/06/01 20:42:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/01 20:42:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Web UI available at http://172.27.220.32:4041
SparkContext available as 'sc' (version = 3.3.1, master = local[*], app id = local-1685648554059)
SparkSession available as 'spark'


In [2]:
path = "./model_data/part-00000-8f3d2c8d-e21b-4ae6-a337-4cbd28bb3ac8-c000.csv"
df = spark.read.option('header' , 'True').csv(path)
df = df.withColumn("Prize",df.Prize.cast('integer'))
df = df.withColumn("nSets",df.nSets.cast('integer'))
df = df.withColumn("Year",df.Year.cast('integer'))
df = df.withColumn("rank_dif",df.rank_dif.cast('double'))
df = df.withColumn("loser_home",df.loser_home.cast('int'))
df = df.withColumn("winner_home",df.winner_home.cast('int'))
df = df.withColumn("GameRank_winner",df.GameRank_winner.cast('double'))
df = df.withColumn("GameRank_loser",df.GameRank_loser.cast('double'))
df = df.withColumn("Height_dif",df.Height_dif.cast('double'))
df_pandas = df.toPandas()


In [3]:
setup = setup(df_pandas,target = "nSets",imputation_type= "iterative",fix_imbalance = True, normalize = True,session_id = 123,)

                        Description             Value
0                        Session id               123
1                            Target             nSets
2                       Target type            Binary
3                    Target mapping        2: 0, 3: 1
4               Original data shape       (76321, 12)
5            Transformed data shape       (94409, 31)
6       Transformed train set shape       (71512, 31)
7        Transformed test set shape       (22897, 31)
8                  Numeric features                 8
9              Categorical features                 3
10         Rows with missing values             33.9%
11                       Preprocess              True
12                  Imputation type         iterative
13  Iterative imputation iterations                 5
14        Numeric iterative imputer          lightgbm
15    Categorical iterative imputer          lightgbm
16         Maximum one-hot encoding                25
17                  Encoding

blender and stacker use the following models "gbc,lgbm,lda,ct"

In [4]:
blender =load_model("./blender_v1")
catboost = load_model("./catboost_v1")
gbc =load_model( "./gbc_v1")
lda =load_model("./lda_v1")
lgbm =load_model("./lgbm_v1")
lr = load_model("./lr_v1")
nb = load_model("./nb_v1")
rf = load_model("./rf_v1")
stacker =load_model("./stacker_v1")
blended_top5_auc = load_model("./blended_top5")
blended_top5_acc = load_model("./blended_top5_acc")

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [5]:
models = [blender,catboost,gbc,lda,lgbm,lr,nb,rf,stacker,blended_top5_auc,blended_top5_acc]

In [6]:
j = 0
for i in models:
    print(j)
    predict_model(i)
    j = j + 1

0
               Model  Accuracy     AUC  Recall  Prec.  F1  Kappa    MCC
0  Voting Classifier    0.6683  0.5666       0      0   0  0.017  0.044
1
                 Model  Accuracy     AUC  Recall  Prec.  F1   Kappa     MCC
0  CatBoost Classifier    0.6648  0.5578       0      0   0  0.0206  0.0413
2
                          Model  Accuracy     AUC  ...  F1  Kappa     MCC
0  Gradient Boosting Classifier    0.6692  0.5641  ...   0  0.004  0.0222

[1 rows x 8 columns]
3
                          Model  Accuracy     AUC  ...  F1   Kappa    MCC
0  Linear Discriminant Analysis    0.4999  0.5542  ...   0  0.0661  0.077

[1 rows x 8 columns]
4
                             Model  Accuracy     AUC  ...  F1   Kappa     MCC
0  Light Gradient Boosting Machine    0.6704  0.5643  ...   0  0.0161  0.0514

[1 rows x 8 columns]
5
                 Model  Accuracy     AUC  Recall  Prec.  F1   Kappa     MCC
0  Logistic Regression       0.5  0.5542       0      0   0  0.0658  0.0765
6
         Model  Accu

In [7]:
top_3_auc = [blended_top5_auc, blender,blended_top5_acc]

In [8]:

plot_model(blended_top5_auc, plot = 'confusion_matrix',save = True)
plot_model(blended_top5_auc, plot = 'auc',save = True)

'AUC.png'

In [12]:
plot_model(blender, plot = 'confusion_matrix',save = True)
plot_model(blender, plot = 'auc',save = True)

'AUC.png'

In [13]:
plot_model(blended_top5_acc, plot = 'confusion_matrix',save = True)
plot_model(blended_top5_acc, plot = 'auc',save = True)

'AUC.png'

In [9]:
j = 0
for i in top_3_auc:
    print(j)
    predict_model(i)
    j = j + 1

0
               Model  Accuracy     AUC  Recall  Prec.  F1   Kappa     MCC
0  Voting Classifier    0.6652  0.5662       0      0   0  0.0201  0.0413
1
               Model  Accuracy     AUC  Recall  Prec.  F1  Kappa    MCC
0  Voting Classifier    0.6683  0.5666       0      0   0  0.017  0.044
2
               Model  Accuracy     AUC  Recall  Prec.  F1  Kappa     MCC
0  Voting Classifier    0.6699  0.5701       0      0   0  0.011  0.0412
